In [2]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout, LSTM 
from keras.optimizers import RMSprop 
import numpy as np
import random
import sys

In [3]:
import string

def clean_text(raw_text):
    translator = str.maketrans('', '', string.punctuation)
    return raw_text.translate(translator)

filename = "Trading with Mexico.txt"
raw_text = open(filename, 'r', encoding='utf-8').read().lower()
raw_text = clean_text(raw_text)


In [4]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [5]:
maxlen = 60 
step = 3
chars = sorted(list(set(raw_text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
sentences = []
next_chars = []

for i in range(0, len(raw_text) - maxlen, step):
    sentences.append(raw_text[i: i + maxlen])
    next_chars.append(raw_text[i + maxlen])

print("Number of sequences:", len(sentences))

Number of sequences: 127146


In [7]:
import numpy as np

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [8]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

2025-06-12 14:56:36.844866: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
import sys
import random
import numpy as np

# Dictionary to store outputs: {epoch: {temperature: generated_text}}
generated_texts = {}

temperatures = [0.2, 0.5, 1.0, 1.2]
epochs = 60

for epoch in range(1, epochs + 1):
    print(f"\n{'='*40}\nEpoch {epoch}\n{'='*40}")
    
    model.fit(X, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(raw_text) - maxlen - 1)
    sentence = raw_text[start_index: start_index + maxlen]
    generated = sentence

    generated_texts[epoch] = {}  # Store texts per epoch

    for temp in temperatures:
        sys.stdout.write(f"\n--- Temperature: {temp}\n")
        sys.stdout.write(f'Seed: "{sentence}"\n')
        
        gen_text = sentence  # Start with the seed

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature=temp)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char
            gen_text += next_char

        generated_texts[epoch][temp] = gen_text

with open("generated_outputs.txt", "w", encoding="utf-8") as f:
    for epoch in sorted(generated_texts.keys()):
        f.write(f"\n{'='*50}\nEpoch {epoch}\n{'='*50}\n")
        for temp, text in generated_texts[epoch].items():
            f.write(f"\n--- Temperature: {temp}\n{text}\n")



Epoch 1


2025-06-12 09:56:55.431717: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 442468080 exceeds 10% of free system memory.


994/994 ━━━━━━━━━━━━━━━━━━━━ 137s 135ms/step - loss: 2.8678

--- Temperature: 0.2
Seed: "nteed bonds of bankrupt state governments but
most of such p"

--- Temperature: 0.5
Seed: "he the the and the the the the the the the the the the the t"

--- Temperature: 1.0
Seed: "he wore the promeve the and ap the the werit he areen of por"

--- Temperature: 1.2
Seed: "nss arf fure f cacedas thw 0acap anrerye
chome catiel of and"

Epoch 2
  1/994 ━━━━━━━━━━━━━━━━━━━━ 1:52 113ms/step - loss: 2.2514

2025-06-12 10:00:39.609342: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 442468080 exceeds 10% of free system memory.


994/994 ━━━━━━━━━━━━━━━━━━━━ 93s 93ms/step - loss: 2.2394

--- Temperature: 0.2
Seed: " us

the result is that today we have little hold on the tra"

--- Temperature: 0.5
Seed: "d of the and and and and and the procan and the wore the whe"

--- Temperature: 1.0
Seed: "ace and erems of the the fion l in the prictows the prating "

--- Temperature: 1.2
Seed: "ed ic thos
thoo  nhtorred ondeves mexica int wroll
the dorud"

Epoch 3
  1/994 ━━━━━━━━━━━━━━━━━━━━ 1:55 117ms/step - loss: 2.1920

...

994/994 ━━━━━━━━━━━━━━━━━━━━ 93s 94ms/step - loss: 2.0907

--- Temperature: 0.2
Seed: "he
previously contented “industrial workers” railway men por"

--- Temperature: 0.5
Seed: "and mexico the mexican of the proment of the wate the mexica"

--- Temperature: 1.0
Seed: "of the and of the greins the prome of the ground mexican the"

--- Temperature: 1.2
Seed: "ould wiskrct ot
the ma5res mexucary the venitivesice thay d "

Epoch 4
  1/994 ━━━━━━━━━━━━━━━━━━━━ 1:49 110ms/step - loss: 2.2572

2025-06-12 10:07:36.882263: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 442468080 exceeds 10% of free system memory.


994/994 ━━━━━━━━━━━━━━━━━━━━ 89s 90ms/step - loss: 1.9976

--- Temperature: 0.2
Seed: "r diaz or huerta or who was not known to be a carranzista
be"

--- Temperature: 0.5
Seed: " mexico and the mexican and the propert of the proment of th"

--- Temperature: 1.0
Seed: "latican but the covale the deas and the inderomation of the "

--- Temperature: 1.2
Seed: "fill
did of leag
 azain vindel anandidered nopew is mexoiac
"

Epoch 5
  1/994 ━━━━━━━━━━━━━━━━━━━━ 1:53 114ms/step - loss: 2.0295

2025-06-12 10:10:34.701521: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 442468080 exceeds 10% of free system memory.


994/994 ━━━━━━━━━━━━━━━━━━━━ 90s 90ms/step - loss: 1.9227

--- Temperature: 0.2
Seed: "her shores touched by more waters in proportion to her area "

--- Temperature: 0.5
Seed: "really the proment of the rean which sean the propert of the"

--- Temperature: 1.0
Seed: "nt of the perical mast be the revolution of of compante the "

--- Temperature: 1.2
Seed: " rincess cound th we lesenct oc alincal on the theres ncam t"

Epoch 6
994/994 ━━━━━━━━━━━━━━━━━━━━ 90s 90ms/step - loss: 1.8674

--- Temperature: 0.2
Seed: "ders
of the proletariat and more important still was said to"

--- Temperature: 0.5
Seed: "he mexican of the interest of the country the marien and the"

--- Temperature: 1.0
Seed: " pare that conter in mexico of the compente but the erolling"

--- Temperature: 1.2
Seed: "g the ravilucablorough an the forlds
comtante whraaed be it "

Epoch 7
994/994 ━━━━━━━━━━━━━━━━━━━━ 89s 89ms/step - loss: 1.8121

--- Temperature: 0.2
Seed: "i have written here something of the magnificence
o

"""
# Summary of Findings on Temperature-based Text Generation

The text generation using LSTM models is greatly influenced by the temperature parameter, which controls the randomness of predictions. During our experiments over 60 epochs with sampling temperatures of [0.2, 0.5, 1.0, 1.2], distinct behaviors in the generated text were observed.

At **temperature=0.2**, the model was highly conservative. The generated sequences were repetitive and deterministic. Often, it kept looping similar words or phrases with little variation. Although the grammar and structure were more coherent, it lacked creativity. This is because lower temperature values make the model strongly favor high-probability characters, essentially removing randomness from sampling.

At **temperature=0.5**, the results showed improvement in diversity while maintaining grammatical consistency. The sentences had more variation and appeared less robotic. It struck a good balance between coherence and novelty, making it one of the optimal values for realistic text generation.

With **temperature=1.0**, the model exhibited significant creativity. The words were more unpredictable and the vocabulary used was broader. However, this came at the cost of occasional grammatical errors or odd word formations. Nevertheless, the model captured some stylistic flair that wasn't present at lower temperatures.

At the extreme end, **temperature=1.2** produced very erratic outputs. The generated text had frequent inconsistencies, non-words, and broke the rules of grammar more often. Still, it offered a glimpse into the depth of the model's learned vocabulary and potential for poetic or abstract writing.

Overall, these results confirm that the temperature setting directly affects how deterministic or creative the output is. A lower temperature makes the model risk-averse and repetitive, while a higher one injects randomness that can lead to both novelty and chaos. For applications like chatbots or fiction writing, a moderate value like 0.5 to 1.0 offers the best trade-off. For generating highly accurate or technical content, lower values may be preferable.

This temperature-tuning experiment emphasizes the importance of sampling strategy in sequence generation and how it allows practitioners to control the behavior of generative models depending on the application domain.
"""